In [1]:
import pandas as pd
import numpy as np

In [4]:
# clusterred words and sequences
cl = pd.read_csv(r"C:\Users\Dima\Downloads\Table  - Sheet1.csv")
#vacancy with number as initial number when scrapped and all requirments
req = pd.read_csv('list_data.csv')
req


,Unnamed: 0,0,1
0,0,"['college degree', 'communication skills', 'dr...",0
1,1,"['delivery driver', 'bicycle courier']",1
2,2,"['head office', 'take place', 'very accurate',...",2
3,3,"['service representative', 'hardware knowledge...",3
4,4,"['customer oriented', 'technical support', 'no...",4
...,...,...,...
981,981,"['employment get', 'social skills', 'care nurs...",995
982,982,"['leading company', 'thinking level', 'work fl...",996
983,983,"['mechanic beton', 'concrete mixer', 'working ...",997
984,984,"['development experience', 'work independently...",998


In [9]:
from nltk.corpus import wordnet


def get_synonims(word):
    synonyms = []
    for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
    return set(synonyms)

# all list of all possible synomims for 2words or 3words
def get_total_list(sentence):
    
    sep_w = sentence.split()
    if len(sep_w)==2:
        f = list(get_synonims(sep_w[0]))
        s=list(get_synonims(sep_w[1]))
        l = [f[i]+" "+s[j] for i in range(len(f)) for j in range(len(s))]
        return l
    else:
        f = list(get_synonims(sep_w[0]))
        s=list(get_synonims(sep_w[1]))
        t = list(get_synonims(sep_w[2]))
        l= [f[i]+" "+s[j]+" "+t[k] for i in range(len(f)) for j in range(len(s)) for k in range(len(t))]
        return l

In [10]:
mapp ={}

# list of current clusters
clusters = [cl[cl['Cluster']==i]['Skills'].iloc[0] for i in range(1,50)]


# here we create a big fat map of all possible skills to our 50 clusteres
for i in clusters:
    w = [cl[cl['Skills']==i]['Words'].iloc[j] for j in range(len(cl[cl['Skills']==i]))]
    for r in w:
        mapp[r]=i
        for s in get_total_list(r):
            mapp[s]=i
      

len(mapp)

156581

# This is a current list of clusters 

In [11]:
clusters

['Accounting',
 'Finance',
 'Commercial',
 'Marketing',
 'HR',
 'Administration',
 'Management',
 'Law',
 'Communication Skills',
 'Leadership',
 'Logistics',
 'Admin Maintenance',
 'Engineer',
 'S.Mechanic',
 'L.Mehanic',
 'Mechanic',
 'Maintainance worker',
 'Technician',
 'Installation',
 'Construction',
 'Software',
 'Programming',
 'IT',
 'ICT',
 'Computer Literacy',
 'Chemistry',
 'Cardiology',
 'Care/Nurse',
 'Dental',
 'GP',
 'Geriatrics',
 'Gynecology',
 'Medical care',
 'Cosmetology',
 'Oncology',
 'Otorhinolaryngology',
 'Pediatrics',
 'Physiotherapy',
 'Psychology/Psychiatrics',
 'Surgeon/Orthopedic',
 'Therapy',
 'Music Therapy',
 'NTH',
 'Cooking',
 'Delivery',
 'Driver',
 'Foreign Language',
 'Teaching',
 'Housing']

In [12]:
mapp.get('leadership communication')

'Communication Skills'

In [13]:
"""just initialise requirments from list of requirments that we generate in another file"""
# I saved as string, so "unstring" here
def get_req(j):

    req_split = req["0"].iloc[j].split(',')
    skills_req=[]
    for i in range(len(req_split)):
        if i ==0:
            skills_req.append(req_split[i][2:-1])
        elif i==len(req_split)-1:
            skills_req.append(req_split[i][2:-2])
        else: 
            skills_req.append(req_split[i][2:-1])
    return skills_req
get_req(66)

['communicative skills',
 'dutch language',
 'administrative position',
 'experienced paralegal',
 'language english',
 'coffee good',
 'have come',
 'law degree',
 'young ambitious',
 'work experience',
 'team player',
 'legal administrative',
 'strong analytical skills']

In [14]:
"""iterate over job requirments to find requirments that are recognised by clusters"""
# basically what clusters are used in a given vacancy
def get_clustered_req(j):
    x = get_req(j)
    true_req = []
    for i in x:
        if mapp.get(i)!=None:
            true_req.append(mapp.get(i))

    return list(set(true_req))


In [15]:
# requirments in terms of clusters for each vacancy
clustered_req=[]
for i in range(len(req)):
    clustered_req.append(get_clustered_req(i))

clustered_req[66]

['Foreign Language', 'Administration', 'Law']

In [10]:
# match skills to requirment
def intersect(skill,requirment):
    x=set(skill)
    y = set(requirment)
    if y.issubset(x):
        return "Available straight away!"
    else:
        return "Skills that have to be obtained " + str(y.difference(y.intersection(x)))
    
intersect(['Foreign Language','IT', 'ICT'], clustered_req[66])

"Skills that have to be obtained {'Administration', 'Law'}"

# Example1

Here x are the skills chosen from the clusters list above

In [19]:
x = ['IT','Management','Programming','Accounting']

print("The entire set of skills for different vacancies:\n")

"""b is responsible for min number of requirments in vacancy we consider - if it's 0,1,2 we ignore the vacancy, a- min number
of intersections beetwen vacancy and skill"""
def skill_is_a_subset_of_req(a=2,b=2):
    indicies = []
    for i in clustered_req:
        
        if sum([i.count(x[j]) for j in range(len(x))])>=max(min(a,len(i)),b) :
            print("For vacancy {ind}".format(ind=clustered_req.index(i)))
            print("All skills required " + str(set(i)))
            print(intersect(x,i)+"\n")
            indicies.append(clustered_req.index(i))
    return indicies
        
indicies = skill_is_a_subset_of_req()

The entire set of skills for different vacancies:

For vacancy 13
All skills required {'Management', 'Administration', 'Accounting'}
Skills that have to be obtained {'Administration'}

For vacancy 29
All skills required {'Accounting', 'IT', 'HR', 'Management'}
Skills that have to be obtained {'HR'}

For vacancy 41
All skills required {'Accounting', 'HR', 'Management'}
Skills that have to be obtained {'HR'}

For vacancy 52
All skills required {'Management', 'IT', 'HR', 'Foreign Language'}
Skills that have to be obtained {'HR', 'Foreign Language'}

For vacancy 103
All skills required {'Driver', 'IT', 'Maintainance worker', 'Management'}
Skills that have to be obtained {'Driver', 'Maintainance worker'}

For vacancy 150
All skills required {'Management', 'Accounting', 'Foreign Language'}
Skills that have to be obtained {'Foreign Language'}

For vacancy 156
All skills required {'Management', 'Delivery', 'Accounting'}
Skills that have to be obtained {'Delivery'}

For vacancy 182
All skills r

In [24]:
true_indicies = [req.iloc[i]["1"] for i in indicies]
print(true_indicies)
Data = pd.read_csv("JobDataTranslated.csv")

#for i in true_indicies:
print(Data['Job Description'][935])
print("")
print("")
print("")

[13, 31, 43, 54, 105, 153, 160, 186, 225, 231, 239, 239, 301, 160, 416, 474, 482, 508, 521, 544, 579, 736, 737, 752, 786, 811, 824, 482, 900, 935, 935, 935]
Project leader Healthcare Technology Care group Meander We are looking for an experienced and creative Project Leader Healthcare Technology. Are you the Project Leader with knowledge of the implementation of (innovative) healthcare technology solutions? Are you enthusiastic about contributing to the development of the digital skills of employees in our organization? Then we are looking for you! Project leader Healthcare technology at Zorggroep Meander At Zorggroep Meander, more than 1,500 employees offer care, treatment and rehabilitation in the East Groningen region. Zorggroep Meander is active at various locations. We have four residential care centers, four residential service centers, a rehabilitation center, district nursing and day care / day treatment in East Groningen. We are there for the vulnerable elderly in the home sit

# Example 2


In [14]:
x = ['S.Mechanic','Engineer','Construction','Maintainance worker']
indicies = skill_is_a_subset_of_req(a=3,b=2)


For vacancy 101
All skills required {'Technician', 'Maintainance worker', 'Driver', 'Engineer', 'Admin Maintenance', 'Mechanic', 'S.Mechanic'}
Skills that have to be obtained {'Driver', 'Mechanic', 'Technician', 'Admin Maintenance'}

For vacancy 212
All skills required {'Engineer', 'Maintainance worker'}
Available straight away!

For vacancy 328
All skills required {'Technician', 'Maintainance worker', 'Driver', 'Engineer', 'S.Mechanic'}
Skills that have to be obtained {'Driver', 'Technician'}

For vacancy 212
All skills required {'Engineer', 'Maintainance worker'}
Available straight away!

For vacancy 669
All skills required {'S.Mechanic', 'Engineer', 'Maintainance worker'}
Available straight away!

For vacancy 731
All skills required {'Maintainance worker', 'Construction'}
Available straight away!

For vacancy 776
All skills required {'Technician', 'Maintainance worker', 'Driver', 'Engineer', 'Care/Nurse', 'Admin Maintenance', 'S.Mechanic'}
Skills that have to be obtained {'Driver', 

# some more text of vacancies:)

In [15]:
true_indicies = [req.iloc[i]["1"] for i in indicies]
Data = pd.read_csv("JobDataTranslated.csv")

for i in true_indicies:
    print(Data['Job Description'][i])
    print("")
    print("")
    print("")

You are going to do this With your technical skills and passion for aircraft technology, you can work as a maintenance technician for an airline. But you can also work as an aircraft mechanic for Defense. Would you like to make sure that our F16's systems work perfectly? Then a job as Aircraft Maintenance Operator (F-16 A1) is all you are looking for. Do you have what it takes? As a Petty Officer Aircraft Maintenance, you maintain the F16 of the Air Force. You come into direct contact with the technology in the aircraft and thus contribute to the Air Force's most important product for carrying out missions: AIRPOWER. What you are going to do depends on where you will be working. For example, you can become crew chief at an F-16 squadron or work at a line or base maintenance or fuel shop. It is also possible to carry out major maintenance or, conversely, the inspection and daily maintenance. You choose!  Note: if you are hired for this position, you will be appointed at the Ministry of 

Note on parameters $a$, $b$: if we set $b$ high >1(2), we will cut off vacancies that aren't processed to well by our algoritm; if the skills set is divergent then high $a$ will make use of vacancies within both categories (so like intraindustry stuff we look for), but if $a$ is low it will also suggest vacancies in one industry. Setting $a$=|entered_skill_set| would mean we look for matches where either a set of skills is a *proper* subset of requirment or another way around